Environment

In [ ]:
import os
import sys
import importlib
if (importlib.util.find_spec("ronek") is None):
  sys.path.append("./../../../")

from ronek import env
env.set(
  device="cpu",
  device_idx=0,
  nb_threads=8,
  floatx="float64"
)

Import libraries

In [ ]:
import torch
import numpy as np

from utils import *

from ronek import const
from ronek.systems import TAFASystem
from matplotlib import pyplot as plt
from silx.io.dictdump import h5todict

Define inputs

In [ ]:
# System
T = 1e4
ic = {
  "cold": [5e2, 1e3, 5e-2],
  "hot":  [3e3, 1e3, 5e-2]
}
tgrid = {
  "lim": [1e-12, 1e-3],
  "pts": 500
}
rom_dims = list(range(8,15,2))
# Paths
paths = {
  "dtb": "./../database/",
  "data": "./data/"
}
# Plotting
saving = True
showing = False

Initialize isothermal master equation model

In [ ]:
model = TAFASystem(
  rates=paths["dtb"] + "/kinetics.hdf5",
  species={
    k: paths["dtb"] + f"/species/{k}.json" for k in ("atom", "molecule")
  },
  use_einsum=False
)
model.update_fom_ops(T)

> Molecule's levels

In [ ]:
gi = model.species["molecule"].lev["g"]
ei = model.species["molecule"].lev['e'] / const.eV_to_J

Balanced POD

In [ ]:
os.makedirs(paths["data"]+"/figs/bases/", exist_ok=True)

In [ ]:
bases = h5todict(paths["data"]+"/bases.hdf5")
s, phi, psi = [bases[k].real for k in ("s", "phi", "psi")]

In [ ]:
# Cumulative energy
cs = 1.0 / np.sum(s**2)
cs *= np.cumsum(s**2)
# Number of principal components
eps = 1e-5
rom_dim = np.where(cs > 1-eps)[0][0]+1
rom_dim

In [ ]:
pltt.cum_energy(
  cs[:rom_dims[-1]],
  figname=paths["data"]+"/figs/bases/cum_en.png",
  save=saving,
  show=showing
)

In [ ]:
for i in range(rom_dims[-1]):
  nb = str(i+1)
  for (name, basis) in (("phi", phi), ("psi", psi)):
    pltt.dist_2d(
      x=ei,
      y=basis[:,i],
      labels=[r"$\epsilon_i$ [eV]", r"$\%s_{%s}$" % (name, nb)],
      scales=["linear", "linear"],
      figname=paths["data"] + f"/figs/bases/{name}_{nb.zfill(2)}.png",
      save=saving,
      show=showing
    )

FOM and ROM solutions

In [ ]:
t = get_tgrid(tgrid["lim"], tgrid["pts"])
i = np.argwhere(t >= 1e-9)[0][0]
j = np.argwhere(t >= 1e-3)[0][0]

In [ ]:
for (name, (T, p, Xa)) in ic.items():
  y0 = get_y0(model, T, p, Xa)
  yfom = solve_fom(model, t, *y0)
  for r in rom_dims:
    yrom = solve_rom(model, t, *y0, phi, psi, r)
    # Postprocessing
    path = paths["data"]+f"/figs/sol/{name}_r{r}/"
    os.makedirs(path, exist_ok=True)
    plot_moments(
      path,
      t[i:j],
      yfom[1][:,i:j],
      yrom[1][:,i:j],
      ei.reshape(-1,1),
      max_mom=11
    )

In [ ]:
def get_tgrid(t_lim, num):
  t = np.geomspace(*t_lim, num=num-1)
  t = np.insert(t, 0, 0.0)
  return t

def get_y0(T, p, Xa):
  n = p / (const.UKB * T)
  na = np.array([n * Xa]).reshape(-1)
  qm = sp_mol.q_int(T)
  nm = n * (1-Xa) * qm / np.sum(qm)
  return na, nm

def compute_fom(t, na_0, nm_0):
  y0 = np.concatenate([na_0, nm_0])
  y = model.solve(t, y0, ops=model.fom_ops, rtol=1e-7, atol=0.0)
  return y[:1], y[1:]

def compute_rom(t, na_0, nm_0, r=5):
  # Update operators
  model.set_basis(phi=phi[:,:r], psi=psi[:,:r])
  model.update_rom_ops()
  # Solve
  y0 = np.concatenate([na_0, model.psi.T @ nm_0])
  y = model.solve(t, y0, ops=model.rom_ops, rtol=1e-7, atol=0.0)
  return y[:1], model.phi @ y[1:]

In [ ]:
t = np.geomspace(1e-12, 1e-3, num=199)
t = np.insert(t, 0, 0.0)

In [ ]:
n_0 = ic["p"] / (const.UKB * ic["T"])
na_0 = np.array([n_0 * ic["Xa"]]).reshape(-1)
qm_0 = sp_mol.q_int(ic["T"])
nm_0 = n_0 * (1.0-ic["Xa"]) * qm_0 / np.sum(qm_0)

In [ ]:
y0 = np.concatenate([na_0, nm_0])
yf = model.solve(t, y0, ops=model.fom_ops, rtol=1e-7, atol=0.0)
na, nm = yf[:1], yf[1:]

In [ ]:
s[:5]

In [ ]:
phi[:,:5].imag

ROM Model - Testing

In [ ]:
rom_dim = 10

In [ ]:
model.set_basis(phi=phi[:,:rom_dim], psi=psi[:,:rom_dim])
model.update_rom_ops()

In [ ]:
model.psi.T @ nm_0

In [ ]:
y0 = np.concatenate([na_0, model.psi.T @ nm_0])
yr = model.solve(t, y0, ops=model.rom_ops, rtol=1e-7, atol=0.0)
na_pred = yr[:1]
nm_pred = model.phi @ yr[1:]

FOM vs. ROM

In [ ]:
100 * np.mean(np.abs(na_pred - na) / np.abs(na))

In [ ]:
plt.loglog(t, na.squeeze())
plt.loglog(t, na_pred.squeeze(), ls='--')

In [ ]:
one = np.ones_like(ei)

In [ ]:
m = nm.T @ one
m_pred = nm_pred.T @ one
100 * np.mean(np.abs(m_pred - m) / np.abs(m))

In [ ]:
plt.loglog(t, m.squeeze())
plt.loglog(t, m_pred.squeeze(), ls='--')

In [ ]:
e = nm.T @ ei / m
e_pred = nm_pred.T @ ei / m_pred
100 * np.mean(np.abs(e_pred - e) / np.abs(e))

In [ ]:
45**3/10**3

In [ ]:
plt.loglog(t, e.squeeze())
plt.loglog(t, e_pred.squeeze(), ls='--')

In [ ]:
# Plot distributions
for i in range(0,200,30):
  print("t = ",t[i])
  plot_2D(
    x=ei,
    y_true=nm[:,i] / sp_mol.lev["g"],
    y_pred=nm_pred[:,i] / sp_mol.lev["g"],
    scales=["linear", "log"],
    filename=paths["data"] + f"/figs/sol_2d_i{str(i).zfill(4)}_rom.png",
    save=saving,
    show=showing
  )

In [ ]:
i = np.argwhere(t >= 1e-9)[0][0]
j = np.argwhere(t >= 1e-4)[0][0]
y = {
  "FOM": nm.T[i:j] / gi,
  "ROM": nm_pred.T[i:j] / gi
}

In [ ]:
# animate(
#   t=t[i:j],
#   x=ei,
#   y=y,
#   frames=50,
#   fps=10,
#   filename=paths["data"] + f"/figs/lev_dist_T0{int(ic['T'])}K_r{rom_dim}.gif",
#   save=saving,
#   show=showing
# )